In [ ]:
# Author: Anik Halder (ahalder@usm.lmu.de)

import numpy as np
import healpy as hp
import os

# Convert shear catalog to healpy map

In [ ]:
def RaDec2pixel(ra, dec, nside):
    #converts ra dec to healpy pixel index 
    #assumes that input ra and dec are in radians
    return hp.pixelfunc.ang2pix(nside, np.pi/2.0 + np.radians(-dec), np.radians(ra))

def create_healpy_map(ra, dec, g1, g2, nside):
    
    npix = hp.nside2npix(nside)
    g1_map = np.zeros(npix)
    g2_map = np.zeros(npix)
    w_map = np.zeros(npix)
    footprint_map = np.zeros(npix)
    
    pix = RaDec2pixel(ra,dec,nside=nside)
    unique_pix, idx, idx_rep = np.unique(pix, return_index=True, return_inverse=True)
    w_map[unique_pix] += np.bincount(idx_rep, weights=w)
    g1_map[unique_pix] += np.bincount(idx_rep, weights=g1*w)
    g2_map[unique_pix] += np.bincount(idx_rep, weights=g2*w)
    mask_map = w_map != 0. # this is footprint of the map (in this bin)
    
    footprint_map[mask_map] = 1
    g1_map[mask_map] =  g1_map[mask_map] / w_map[mask_map]  
    g2_map[mask_map] =  g2_map[mask_map] / w_map[mask_map]  
    
    return g1_map, g2_map, w_map, footprint_map

In [ ]:
n_g = '0.6'
catpath = '/global/homes/j/jharno/IA-infusion/SkySim5000/GalCat/StageIV_nz/V0/'
NSIDE = 2048

filepath_map_data = './data/' # the healpy maps will be stored in this path

if (os.path.isdir(filepath_output) == False):
    os.makedirs(filepath_output)

for i in range(5):
    tomo_bin = 'tomo'+str(i+1)
    catname = catpath+'/GalCat_'+tomo_bin+'_All_'+n_g+'GpAM_RA_Dec_g1_g2_w.asc'
    
    cat = np.loadtxt(catname)
    ra, dec, g1, g2, w, z = cat.T
    
    g1_map, g2_map, w_map, footprint_map = create_healpy_map(ra, dec, g1, g2, NSIDE)
    
    hp.write_map(filepath_map_data+'GalMap_'+tomo_bin+'_All_'+n_g+'GpAM_nside_'+str(NSIDE)+'_g1.fits', g1_map, overwrite=True)
    hp.write_map(filepath_map_data+'GalMap_'+tomo_bin+'_All_'+n_g+'GpAM_nside_'+str(NSIDE)+'_g2.fits', g2_map, overwrite=True)
    hp.write_map(filepath_map_data+'GalMap_'+tomo_bin+'_All_'+n_g+'GpAM_nside_'+str(NSIDE)+'_w.fits', w_map, overwrite=True)
    hp.write_map(filepath_map_data+'GalMap_'+tomo_bin+'_All_'+n_g+'GpAM_nside_'+str(NSIDE)+'_footprint.fits', footprint_map, overwrite=True)